In [20]:
from scapy.all import rdpcap, UDP, IP
import datetime
import re
from tqdm import tqdm

def analyse_experiment(pcap_file, entry_file):

    packets = rdpcap(pcap_file)

    entry_timestamps = []
    packet_timestamps = []

    for pkt in tqdm(packets):
        if IP in pkt and UDP in pkt:
            if pkt[IP].dst == "10.0.0.1":
                ts = pkt.time
                ts = float(pkt.time)
                sec = int(ts)
                nsec = int((ts - sec) * 1_000_000_000)

                formatted = f"{datetime.datetime.fromtimestamp(sec).strftime('%H:%M:%S')}.{nsec:09d}"
                packet_timestamps.append(formatted)

    with open(entry_file) as f:
        lines = f.readlines()
        add_lines = []
        timestamps = []
        for line in tqdm(lines):
            if "Adding entry at:" in line:
                add_lines.append(line.strip())
        for line in tqdm(add_lines):
            match = re.search(r"Adding entry at:\s+(\d{2}:\d{2}:\d{2}\.\d+)", line)
            if match:
                timestamp = match.group(1)
                #print("Extracted timestamp:", timestamp)
                timestamps.append(timestamp)
            else:
                print("No timestamp found")
        #print(timestamps)
        entry_timestamps = timestamps
    def to_ns_timestamp(tstr):
        hh, mm, rest = tstr.split(":")
        ss, ns = rest.split(".")
        hh = int(hh)
        mm = int(mm)
        ss = int(ss)
        ns = int(ns)
        total_ns = ((hh*3600 + mm*60 + ss) * 1_000_000_000) + ns
        return total_ns


    entry_ns = [to_ns_timestamp(t) for t in entry_timestamps]
    packet_ns = [to_ns_timestamp(t) for t in packet_timestamps]

    packet_ns.sort()

    latencies = []

    for entry_time in entry_ns:
        first_packet_after_entry = None

        for packet_time in packet_ns:
            if packet_time > entry_time:
                first_packet_after_entry = packet_time
                break

        if first_packet_after_entry is None:
            latencies.append(None)
            continue

        latency = first_packet_after_entry - entry_time
        latencies.append(latency)

    latencies_cleaned = [l for l in latencies if l is not None]
    print(f"\nDurchschnittliche Latenz: {sum(latencies_cleaned)/len(latencies_cleaned)/1_000:.3f} ns")
    #print(f"Maximale Latenz: {max(latencies_cleaned):.3f} microseconds")
    print(f"Minimale Latenz: {min(latencies_cleaned)/1_000:.3f} ns")

In [21]:
analyse_experiment("measurements/test1.pcap", "measurements/entries_1.txt")

100%|██████████| 89/89 [00:00<00:00, 754127.39it/s]



Durchschnittliche Latenz: 1857.539 ns
Minimale Latenz: 207.612 ns


In [22]:
analyse_experiment("measurements/test2.pcap", "measurements/entries_2.txt")

100%|██████████| 304/304 [00:00<00:00, 1031608.75it/s]



Durchschnittliche Latenz: 2243.204 ns
Minimale Latenz: 99.985 ns


In [23]:
analyse_experiment("measurements/test4.pcap", "measurements/entries_4.txt")

100%|██████████| 30/30 [00:00<00:00, 365782.33it/s]



Durchschnittliche Latenz: 1736.784 ns
Minimale Latenz: 1.098 ns


In [24]:
analyse_experiment("measurements/test5.pcap", "measurements/entries_5.txt")

100%|██████████| 91/91 [00:00<00:00, 739693.15it/s]



Durchschnittliche Latenz: 672.426 ns
Minimale Latenz: 0.055 ns


In [25]:
analyse_experiment("measurements/test6.pcap", "measurements/entries_6.txt")

100%|██████████| 87/87 [00:00<00:00, 740171.29it/s]



Durchschnittliche Latenz: 1333.965 ns
Minimale Latenz: 387.479 ns


In [26]:
analyse_experiment("measurements/test7.pcap", "measurements/entries_7.txt")

100%|██████████| 90/90 [00:00<00:00, 798070.53it/s]



Durchschnittliche Latenz: 1318.602 ns
Minimale Latenz: 495.565 ns


In [27]:
analyse_experiment("measurements/test8.pcap", "measurements/entries_8.txt")

100%|██████████| 89/89 [00:00<00:00, 717871.26it/s]



Durchschnittliche Latenz: 1325.309 ns
Minimale Latenz: 608.570 ns


In [28]:
analyse_experiment("measurements/test9.pcap", "measurements/entries_9.txt")

100%|██████████| 90/90 [00:00<00:00, 758006.75it/s]



Durchschnittliche Latenz: 1515.539 ns
Minimale Latenz: 669.060 ns


In [29]:
analyse_experiment("measurements/test10.pcap", "measurements/entries_10.txt")

100%|██████████| 96/96 [00:00<00:00, 820067.58it/s]



Durchschnittliche Latenz: 1615.180 ns
Minimale Latenz: 581.377 ns
